In [ ]:
from textblob import TextBlob 
import tweepy
import matplotlib.pyplot as plt
from tweepy import OAuthHandler

def percentage(part, whole):
    return 100*float(part)/float(whole)

consumerKey="SyoUZ9Wj3sK8okiOWYLWxEBig" 
consumerSecret="7t6Zm1W6Zg27mGqtaMaHgQcJIHPTnwVqi6p4OdgOmHcM8RXTxk"
accessToken="1184299756232826880-8HxGKKygpfb6wJB9xumKnezUdBdkUF"
accessTokenSecret="8QGA3DZVGUbrk2Hp87aGYoGIhZDXJFYvdQFiL0F6pGx0j"

auth=tweepy.OAuthHandler(consumerKey,consumerSecret)
auth.set_access_token(accessToken,accessTokenSecret)
api=tweepy.API(auth)

searchTerm=input("Enter keyword/hashtag to search about:")
noOfSearchTerms=int(input("Enter how many tweets to analyse:"))

tweets=tweepy.Cursor(api.search, q=searchTerm,lang="English").items(noOfSearchTerms)

positive=0
negative=0 
neutral=0
polarity=0

for tweet in tweets:
    analysis=TextBlob(tweet.text)
    polarity+=analysis.sentiment.polarity
    
    if(analysis.sentiment.polarity==0):
        neutral+=1
        
    elif(analysis.sentiment.polarity<0.00):
        negative+=1
    elif(analysis.sentiment.polarity>0.00):
        positive+=1
        
positive=percentage(positive,noOfSearchTerms)
negative=percentage(negative,noOfSearchTerms)
neutral=percentage(neutral,noOfSearchTerms)
polarity=percentage(polarity,noOfSearchTerms)
        
positive=format(positive,'.2')
negative=format(positive,'.2')
neutral=format(positive,'.2')

print("How people are reacting on"+searchTerm+"by analyzing"+str(noOfSearchTerms)+"Tweets.")

if(polarity==0):
    print("Neutral")
elif(polarity<0.00):
    print("Negative")
elif(polarity>0.00):
    print("Positive")
    
labels=['Positive['+str(positive)+'%]','Neutral['+str(neutral)+'%]','Negative['+str(negative)+'%]']
sizes=[positive,neutral,negative]
colors=['yellowgreen','gold','red']
patches,texts=plt.pie(sizes,colors=colors,startangle=90)
plt.legend(patches,labels,loc="best")
plt.title("How people are reacting on"+" "+searchTerm+" "'by analyzing'+str(noOfSearchTerms)+'Tweets.')
plt.axis('equal')
plt.tight_layout()
plt.show();
    

In [ ]:

import tweepy
import json
from pymongo import MongoClient

class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def on_connect(self):
        """Called when the connection is made"""
        
        print("You're connected to the streaming server.")

    def on_error(self, status_code):
        """This is called when an error occurs"""
        
        print('Error: ' + repr(status_code))
        return False

    def on_data(self, data):
        """This will be called each time we receive stream data"""
        
        client = MongoClient()

        # I stored the tweet data in a database called 'training_tweets' in MongoDB, if 
        # 'training_tweets' does not exist it will be created for you.
        db = client.training_tweets

        # Decode JSON
        datajson = json.loads(data)

        # I'm only storing tweets in English. I stored the data for these tweets in a collection 
        # called 'mining_tweets' of the 'training_tweets' database. If 
        # 'mining_tweets' does not exist it will be created for you. 
        if "lang" in datajson and datajson["lang"] == "en":
            db.mine_tweets.insert_one(datajson)


if __name__ == "__main__":

    # These are provided to you through the Twitter API after you create a account
    consumer_key = "SyoUZ9Wj3sK8okiOWYLWxEBig"
    consumer_secret = "7t6Zm1W6Zg27mGqtaMaHgQcJIHPTnwVqi6p4OdgOmHcM8RXTxk"
    access_token = "1184299756232826880-8HxGKKygpfb6wJB9xumKnezUdBdkUF"
    access_token_secret = "8QGA3DZVGUbrk2Hp87aGYoGIhZDXJFYvdQFiL0F6pGx0j"

    auth1 = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth1.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the 
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box. 
    LOCATIONS = [-4.8995204,33.9098987, 4.62,41.899578]    # Nairobi, Kenya
    
    WORDS = ['#Ebola', '#cancer', '#malaria','#AIDS','#corona_virus']

    stream_listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True))
    stream = tweepy.Stream(auth=auth1, listener=stream_listener)
    stream.filter(track=WORDS)
    stream.filter(locations=LOCATIONS)
